In [59]:
from nela_features.nela_features import NELAFeatureExtractor
import numpy as np
import pandas as pd
import re
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split 

In [43]:
fake_data=pd.read_csv("dataset/Fake.csv")
real_data=pd.read_csv("dataset/True.csv")
nela = NELAFeatureExtractor()

In [44]:
def filter_short(df):
    df["text"] = df.text.astype(str)
    df["WordsCount"]=df["text"].apply(lambda x: len(re.sub('[^a-zA-Z]', ' ', x)))
    df=df[(df["WordsCount"]>= 5)]
    if 'id' in df.columns:
        df = df[df['id'].str.isnumeric()]
    df = df[df['text'].str.isnumeric() == False]
    df = df[df['text'] != None]
    df = df[df['text'] != ' ']
    df = df.drop(["WordsCount"], axis=1)
    return df


fake_data["label"]="fake"
fake_data = filter_short(fake_data)
real_data["label"]="real"
real_data = filter_short(real_data)

real_data = real_data.sample(1000)
fake_data = fake_data.sample(1000)


final_data= pd.concat([fake_data,real_data])

final_data = final_data.drop(["subject","date"], axis=1)
final_data=final_data[["text","label"]]
final_data['label'] = final_data['label'].map({'real':1, 'fake':0})

In [45]:
final_data = final_data.values

In [46]:
def just_vector(text):
    feature_vector, feature_names = nela.extract_all(text)
    return feature_vector

In [47]:
final_data = [[just_vector(x),y] for x,y in tqdm(final_data)]

  0%|          | 0/2000 [00:00<?, ?it/s]

C:\Users\Julian\AppData\Roaming\Python\Python310\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\Julian\AppData\Roaming\Python\Python310\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\Julian\AppData\Roaming\Python\Python310\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will r

In [56]:
data = [x[0]+[x[1]] for x in final_data]


In [57]:
feature_vector, feature_names = nela.extract_all("hello")
df = pd.DataFrame(data, columns=feature_names+['label'])

In [62]:
X_train, X_test, Y_train, Y_test = train_test_split([x[0] for x in final_data], [x[1] for x in final_data], stratify = [x[1] for x in final_data])

In [65]:
train = [x[0]+[x[1]] for x in list(zip(X_train,Y_train))]

In [67]:
test = [x[0]+[x[1]] for x in list(zip(X_test,Y_test))]

In [68]:
train_df = pd.DataFrame(train, columns=feature_names+['label'])
test_df = pd.DataFrame(test, columns=feature_names+['label'])

In [69]:
train_df.to_csv("dataset/fakenewstrain.csv")
test_df.to_csv("dataset/fakenewstest.csv")